# Environment 2

In [1]:
import pandas as pd
import numpy as np
import requests
from time import sleep
from keras.layers import Input, Embedding, Flatten, Dot, Concatenate, Dense
from keras.models import Model

Using TensorFlow backend.


In [2]:
user_id = '0EFZ3W3H6IQRD9DDSPS2'
base_url = 'http://35.180.254.42'
url_reset = base_url + '/reset'
url_predict = base_url + '/predict'
params = {'user_id': user_id}

In [3]:
# json file
r = requests.get(url=url_reset, params=params)
data = r.json()

In [4]:
data.keys()

dict_keys(['item_history', 'nb_items', 'nb_users', 'next_item', 'next_user', 'next_variables', 'rating_history', 'user_history', 'variables_history'])

In [5]:
nb_items = data['nb_items']
nb_users = data['nb_users']
nb_variables = len(data['variables_history'][0])

user_history = data['user_history']
item_history = data['item_history']
rating_history = data['rating_history']
variables_history = np.array(data['variables_history'])

next_user = data['next_user']
next_item = data['next_item']
next_variables = data['next_variables']

## Model 1

In [6]:
 # For each sample we input the integer identifiers
# of a single user and a single item
user_id_input = Input(shape=[1], name='user')
item_id_input = Input(shape=[1], name='item')
variable_input = Input(shape=[5], name='variable')

embedding_size = 30
user_embedding = Embedding(output_dim=embedding_size, input_dim=nb_users + 1,
                           input_length=1, name='user_embedding')(user_id_input)

item_embedding = Embedding(output_dim=embedding_size, input_dim=nb_items + 1,
                           input_length=1, name='item_embedding')(item_id_input)

# reshape from shape: (batch_size, input_length, embedding_size)
# to shape: (batch_size, input_length * embedding_size) which is
# equal to shape: (batch_size, embedding_size)
user_vecs = Flatten()(user_embedding)
item_vecs = Flatten()(item_embedding)

cc = Concatenate()([user_vecs, item_vecs, variable_input])

y = Dense(20, activation='relu')(cc)

yy = Dense(1, activation='relu')(y)

model = Model(inputs=[user_id_input, item_id_input, variable_input], outputs=yy)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
item (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
user_embedding (Embedding)      (None, 1, 30)        3030        user[0][0]                       
__________________________________________________________________________________________________
item_embedding (Embedding)      (None, 1, 30)        9030        item[0][0]                       
__________________________________________________________________________________________________
flatten_1 

In [7]:
model.compile(optimizer='adamax', loss='mean_squared_error')

# Training the model
history = model.fit([user_history, item_history, variables_history], rating_history,
                    batch_size=64, epochs=80, validation_split=0.1,
                    shuffle=True)

Train on 1800 samples, validate on 200 samples
Epoch 1/80
1800/1800 [==============================] - 0s 144us/step - loss: 8.5952 - val_loss: 6.1603
Epoch 2/80
1800/1800 [==============================] - 0s 25us/step - loss: 4.9797 - val_loss: 3.3033
Epoch 3/80
1800/1800 [==============================] - 0s 28us/step - loss: 2.9951 - val_loss: 2.3983
Epoch 4/80
1800/1800 [==============================] - 0s 22us/step - loss: 2.5073 - val_loss: 2.1420
Epoch 5/80
1800/1800 [==============================] - 0s 25us/step - loss: 2.2146 - val_loss: 1.8997
Epoch 6/80
1800/1800 [==============================] - 0s 28us/step - loss: 1.9013 - val_loss: 1.6447
Epoch 7/80
1800/1800 [==============================] - 0s 25us/step - loss: 1.5990 - val_loss: 1.4193
Epoch 8/80
1800/1800 [==============================] - 0s 19us/step - loss: 1.3375 - val_loss: 1.2477
Epoch 9/80
1800/1800 [==============================] - 0s 28us/step - loss: 1.1281 - val_loss: 1.1183
Epoch 10/80
1800/1800 [==

1800/1800 [==============================] - 0s 30us/step - loss: 0.3548 - val_loss: 0.6591
Epoch 80/80
1800/1800 [==============================] - 0s 19us/step - loss: 0.3526 - val_loss: 0.6653


In [8]:
nb_samples = 100
mse, mae = 0, 0
prediction = 3 # baseline

for i in range(nb_samples):
    sleep(0.05)
    r = requests.get(url=url_predict, params=params)
    d = r.json()

    # error for last prediction
    rating = d['rating']
    mse += (rating - prediction) ** 2
    mae += np.abs(rating - prediction)
    
    # predict next
    next_user = d['next_user']
    next_item = d['next_item']
    next_variables = d['next_variables']
    prediction = model.predict([[next_user], [next_item], [next_variables]])[0][0]

print('mse: ', mse/nb_samples)
print('mae: ', mae/nb_samples)

mse:  0.7429839905947659
mae:  0.677650551199913


------------------------------------

## Model 2

In [10]:
 # For each sample we input the integer identifiers
# of a single user and a single item
user_id_input = Input(shape=[1], name='user')
item_id_input = Input(shape=[1], name='item')
variable_input = Input(shape=[5], name='variable')

embedding_size = 30
user_embedding = Embedding(output_dim=embedding_size, input_dim=nb_users + 1,
                           input_length=1, name='user_embedding')(user_id_input)

item_embedding = Embedding(output_dim=embedding_size, input_dim=nb_items + 1,
                           input_length=1, name='item_embedding')(item_id_input)

# reshape from shape: (batch_size, input_length, embedding_size)
# to shape: (batch_size, input_length * embedding_size) which is
# equal to shape: (batch_size, embedding_size)
user_vecs = Flatten()(user_embedding)
item_vecs = Flatten()(item_embedding)

y = Dot(axes=1)([user_vecs, item_vecs])

# cc = Concatenate()([y, variable_input])

# yy = Dense(1, activation='relu')(cc)

model = Model(inputs=[user_id_input, item_id_input, variable_input], outputs=y)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
item (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
user_embedding (Embedding)      (None, 1, 30)        3030        user[0][0]                       
__________________________________________________________________________________________________
item_embedding (Embedding)      (None, 1, 30)        9030        item[0][0]                       
__________________________________________________________________________________________________
flatten_3 

In [11]:
model.compile(optimizer='adamax', loss='mean_squared_error')

# Training the model
history = model.fit([user_history, item_history, variables_history], rating_history,
                    batch_size=64, epochs=60, validation_split=0.1,
                    shuffle=True)

Train on 1800 samples, validate on 200 samples
Epoch 1/60
1800/1800 [==============================] - 0s 97us/step - loss: 10.1307 - val_loss: 10.1922
Epoch 2/60
1800/1800 [==============================] - 0s 15us/step - loss: 10.1162 - val_loss: 10.1913
Epoch 3/60
1800/1800 [==============================] - 0s 14us/step - loss: 10.1051 - val_loss: 10.1900
Epoch 4/60
1800/1800 [==============================] - 0s 13us/step - loss: 10.0930 - val_loss: 10.1881
Epoch 5/60
1800/1800 [==============================] - 0s 14us/step - loss: 10.0792 - val_loss: 10.1851
Epoch 6/60
1800/1800 [==============================] - 0s 13us/step - loss: 10.0632 - val_loss: 10.1810
Epoch 7/60
1800/1800 [==============================] - 0s 14us/step - loss: 10.0447 - val_loss: 10.1755
Epoch 8/60
1800/1800 [==============================] - 0s 15us/step - loss: 10.0231 - val_loss: 10.1674
Epoch 9/60
1800/1800 [==============================] - 0s 12us/step - loss: 9.9981 - val_loss: 10.1576
Epoch 10/

In [12]:
nb_samples = 100
mse, mae = 0, 0
prediction = 3 # baseline

for i in range(nb_samples):
    sleep(0.05)
    r = requests.get(url=url_predict, params=params)
    d = r.json()

    # error for last prediction
    rating = d['rating']
    mse += (rating - prediction) ** 2
    mae += np.abs(rating - prediction)
    
    # predict next
    next_user = d['next_user']
    next_item = d['next_item']
    next_variables = d['next_variables']
    prediction = model.predict([[next_user], [next_item], [next_variables]])[0][0]

print('mse: ', mse/nb_samples)
print('mae: ', mae/nb_samples)

mse:  4.866380674852802
mae:  1.8361245712637901
